In [1]:
import numpy as np
import pandas as pd
import yfinance as yf


In [2]:

# Define the stock symbols
symbols = {
    "NFLX": "NFLX",
    "BA": "BA",
    "T": "T",
    "MGM": "MGM",
    "TSLA": "TSLA",
    "sp500": "^GSPC"
}

# Define the date range
start_date = "2013-01-01"
end_date = "2023-01-01"

# Fetch the data
data = {}
for symbol, ticker in symbols.items():
    data[symbol] = yf.download(ticker, start=start_date, end=end_date)["Close"]


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:

# Create a DataFrame
df = pd.DataFrame(data)

# Reset the index to have 'Date' as a column
df.reset_index(inplace=True)

# Remove missing values
df.dropna(inplace=True)


In [4]:

# Convert 'Date' to datetime and set it as the index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Resample the data to monthly frequency and take the last observation for each month
monthly_data = df.resample('M').last()

# Calculate the monthly percentage return
monthly_returns = monthly_data.pct_change() * 100

# Remove the market index from returns to focus on individual stocks
stock_returns = monthly_returns.drop(columns=['sp500'])


C:\Users\spars\AppData\Local\Temp\ipykernel_3824\1929050720.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = df.resample('M').last()


In [5]:

# Calculate the standard deviation for each stock
std_devs = stock_returns.std()

# Calculate the beta for each stock
market_returns = monthly_returns['sp500']


In [6]:

# Initialize a dictionary to store beta values
betas = {}

for stock in stock_returns.columns:
    # Calculate covariance of stock with the market
    covariance = stock_returns[stock].cov(market_returns)
    # Calculate variance of the market returns
    market_variance = market_returns.var()
    # Calculate beta
    beta = covariance / market_variance
    betas[stock] = beta


In [7]:

# Convert betas to a DataFrame for better readability
betas_df = pd.DataFrame(list(betas.items()), columns=['Stock', 'Beta'])

# Assume a risk-free rate (annualized, so divide by 12 for monthly)
risk_free_rate = 0.03 / 12  # 3% annual risk-free rate

# Calculate the Treynor Ratio for each stock
treynor_ratios = (stock_returns.mean() - risk_free_rate) / betas_df.set_index('Stock')['Beta']


In [8]:

# Calculate Value at Risk (VaR) at 95% and 99% confidence levels
def calculate_var(returns, confidence_level):
    if returns.empty or len(returns) < 1:
        return np.nan  # Return NaN if there's insufficient data
    return -np.percentile(returns, (1 - confidence_level) * 100)

# Apply VaR calculation with additional checks
var_95 = stock_returns.apply(lambda x: calculate_var(x.dropna(), 0.95))
var_99 = stock_returns.apply(lambda x: calculate_var(x.dropna(), 0.99))


In [9]:

# Combine results into a single DataFrame
results_df = pd.DataFrame({
    'Stock': std_devs.index,
    'Standard Deviation': std_devs.values,
    'Beta': betas_df.set_index('Stock')['Beta'].reindex(std_devs.index).values,
    'Treynor Ratio': treynor_ratios.reindex(std_devs.index).values,
    'VaR 95%': var_95.values,
    'VaR 99%': var_99.values
})

print(results_df)


  Stock  Standard Deviation      Beta  Treynor Ratio    VaR 95%    VaR 99%
0  NFLX           12.480315  1.168477       2.517893  13.030491  27.641347
1    BA           10.671564  1.419744       0.972808  13.331257  24.053124
2     T            5.514140  0.543637      -0.275289   8.719142  13.813483
3   MGM           11.863803  1.913574       0.797990  11.658214  20.258991
4  TSLA           19.204345  1.804237       2.721835  19.308725  22.426234
